### Calculating Beam-ID for all points in PandaSet

In [68]:
import os
import numpy as np
# CONFIGURATION
DATA_PATH = os.path.join('buni','dataset','pandaset')
SCENE_IDX = 3
FRAME_IDX = 70
GROUND_LABELS = np.array([6, 7, 8, 9, 10, 11, 12, 34, 35, 37, 38, 39])
user_home = os.path.expanduser('~') 
dataset_path = os.path.join(user_home,DATA_PATH)

In [ ]:
from pandaset import DataSet
import logging

dataset = DataSet(dataset_path)
scenes_with_semantic_labels = sorted(dataset.sequences(with_semseg=True), key=int)
print(f"List of sequences available with semantic segmentation:\n{scenes_with_semantic_labels}")
scene = dataset[scenes_with_semantic_labels[SCENE_IDX]]
print(f"Selected scene/sequence: {scenes_with_semantic_labels[SCENE_IDX]}")
scene.load_lidar()
scene.load_semseg()
logger = logging.getLogger(__name__)
logging.info(f"Loaded scene {SCENE_IDX} with frame {FRAME_IDX}")
lidar_data = scene.lidar.data[FRAME_IDX]
lidar_poses = scene.lidar.poses[FRAME_IDX]
labels = scene.semseg.data[FRAME_IDX]

from general_utils import pandaset_utils as pdutils
from general_utils import gen_utils

gen_utils.check_type(lidar_data,"lidar_data", logger)
gen_utils.check_type(labels,"labels", logger)

lidar_data, lidar_labels = pdutils.cleanup_lidar_data_and_labels(lidar_data, labels, lidar_poses,logger)

logger.info(f"Shape of lidar_data before reshaping: {lidar_data.shape}")
gen_utils.check_type(lidar_data,"lidar_data",logger)
lidar_points = lidar_data.iloc[:,:3].to_numpy()
lidar_points = lidar_points.astype('float64')
logger.info(f"Shape of lidar_data after reshaping: {lidar_points.shape}")
gen_utils.check_type(lidar_points,"lidar_points",logger)



In [70]:
import numpy as np

# Assuming points is an Nx3 array with columns [x, y, z] representing the point cloud
def assign_beam_ids(points):
    # Calculate elevation for each point
    # Elevation = arctan(z / sqrt(x^2 + y^2))
    x, y, z = points[:, 0], points[:, 1], points[:, 2]
    elevations = np.arctan2(z, np.sqrt(x**2 + y**2))

    # Find max and min elevation values
    min_elevation = np.min(elevations)
    max_elevation = np.max(elevations)

    # Create 64 bins between min and max elevation
    bins = np.linspace(min_elevation, max_elevation, 65)

    # Assign each point to the closest bin
    beam_ids = np.digitize(elevations, bins) - 1
    beam_ids = np.clip(beam_ids, 0, 63)  # Ensure beam IDs are within the range [0, 63]

    return beam_ids
 
# Example usage
# Replace 'points' with your actual point cloud data
# points = np.array([...])  # Nx3 array of points
# beam_ids = assign_beam_ids(points)

import numpy as np

def assign_quantile_beam_ids(points, num_bins=64):
    # Calculate elevations for each point
    x, y, z = points[:, 0], points[:, 1], points[:, 2]
    elevations = np.arctan2(z, np.sqrt(x**2 + y**2))
    
    # Create bins based on quantiles to balance the number of points per bin
    bins = np.percentile(elevations, np.linspace(0, 100, num_bins + 1))
    beam_ids = np.digitize(elevations, bins) - 1
    beam_ids = np.clip(beam_ids, 0, num_bins - 1)

    # Reshape the beam_ids array to have shape (n, 1)
    beam_ids = beam_ids.reshape(-1, 1)

    return beam_ids


In [71]:
# import matplotlib.colors as mcolors
# import open3d as o3d
# def visualize_beam_ids(lidar_points, beam_ids):

#     # Create a point cloud in Open3D and assign colors based on beam IDs
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(lidar_points)

#     # Use a discrete colormap with enough unique colors
#     unique_beam_ids = np.unique(beam_ids)
#     n_unique_beam_ids = len(unique_beam_ids)

#     # Generate a colormap with N unique colors
#     colors_list = list(mcolors.CSS4_COLORS.values())  # Get a large list of CSS colors
#     if n_unique_beam_ids > len(colors_list):
#         raise ValueError("Not enough unique colors available. Increase the color list or choose another colormap.")

#     # Create a color dictionary that maps each beam ID to a unique color
#     beam_id_to_color = {beam_id: colors_list[i % len(colors_list)] for i, beam_id in enumerate(unique_beam_ids)}

#     # Map the beam IDs to the corresponding colors
#     colors = np.array([mcolors.to_rgb(beam_id_to_color[beam_id]) for beam_id in beam_ids])

#     # Set the colors to the point cloud
#     pcd.colors = o3d.utility.Vector3dVector(colors)

#     # Visualize the point cloud
#     o3d.visualization.draw_geometries([pcd])

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import open3d as o3d

def visualize_beam_ids3(lidar_points, beam_ids):
    # Ensure beam_ids has the correct shape
    if beam_ids.shape[1] != 1:
        raise ValueError("Beam IDs must be an (n, 1) array.")
    
    # Reshape beam_ids to be a flat array for color mapping
    beam_ids = beam_ids.flatten()
    
    # Check shapes for debugging
    print("lidar_points shape:", lidar_points.shape)
    print("beam_ids shape after flattening:", beam_ids.shape)

    # Ensure beam_ids has the correct length
    if lidar_points.shape[0] != beam_ids.shape[0]:
        raise ValueError("Error: The number of beam IDs does not match the number of lidar points.")

    # Create a point cloud in Open3D and assign colors based on beam IDs
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(lidar_points)

    # Use the 'hsv' colormap from matplotlib for better distinction of beam IDs
    cmap = plt.get_cmap('hsv')

    # Normalize beam IDs to fit the colormap range
    norm = mcolors.Normalize(vmin=0, vmax=63)  # Assuming beam IDs are in the range [0, 63]

    # Assign colors based on the normalized beam IDs
    colors = cmap(norm(beam_ids))[:, :3]  # Extract RGB channels
    colors = colors.reshape(-1, 3)  # Ensure the shape is (n, 3)

    # Check color array shape for debugging
    print("colors shape:", colors.shape)

    # Set the colors to the point cloud
    pcd.colors = o3d.utility.Vector3dVector(colors)

    # Visualize the point cloud with a larger point size
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)

    # Set rendering options for better visualization
    render_option = vis.get_render_option()
    render_option.point_size = 2.0  # Increase point size for better visibility
    render_option.background_color = np.array([1, 1, 1])  # Set background color to white for contrast

    vis.run()
    vis.destroy_window()

# Example usage
# lidar_points = np.array([[x1, y1, z1], [x2, y2, z2], ...])  # n x 3 array of points
# beam_ids = np.array([[id1], [id2], [id3], ...])  # n x 1 array of beam IDs
# visualize_beam_ids3(lidar_points, beam_ids)

In [ ]:
beam_ids = assign_quantile_beam_ids(lidar_points)

visualize_beam_ids3(lidar_points,beam_ids)

lidar_points_with_beam_ids = np.hstack((lidar_points, beam_ids))


### Ray-Dropping

+ `beam_ids` is a numpy array. 
+ Joint `beam_ids` with `lidar_points` to get `lidar_points_with_beam_ids`

In [ ]:
import numpy as np

def random_beam_drop(points):
    # Randomly select a beam drop ratio from [1, 2, 3]
    beam_drop_ratio = np.random.choice([1, 2, 3])
    # beam_drop_ratio = 3
    print(f"Beam drop ratio: {beam_drop_ratio}")
    # Randomly select a starting beam index
    start_index = np.random.randint(0, beam_drop_ratio)

    # Apply the ray-dropping condition
    mask = (points[:, 3] - start_index) % beam_drop_ratio == 0
    return points[mask]


def spherical_coordinates_conversion(points):
    # Convert to spherical coordinates
    x, y, z = points[:, 0], points[:, 1], points[:, 2]
    radial_dist = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arctan2(y, x)  # azimuth
    phi = np.arcsin(z / radial_dist)  # elevation

    # Filter out points with radial distance < 0.1
    valid_mask = radial_dist > 0.1

    return np.vstack((theta[valid_mask], phi[valid_mask], radial_dist[valid_mask])).T, valid_mask

def random_spherical_drop(points, valid_mask):
    # Sample spherical resolutions for theta and phi
    spherical_resolutions = np.random.choice([600, 900, 1200, 1500])
    
    # Convert theta and phi to grid cells
    spherical_coords, valid_mask = spherical_coordinates_conversion(points)
    theta_grid = (spherical_coords[:, 0] * spherical_resolutions).astype(int)
    phi_grid = (spherical_coords[:, 1] * spherical_resolutions).astype(int)
    
    # Randomly sample spherical drop ratio
    spherical_drop_ratio = np.random.choice([1, 2])
    # spherical_drop_ratio = 2
    print(f"Spherical drop ratio: {spherical_drop_ratio}")
    # Apply the ray-dropping condition in the spherical coordinates
    theta_mask = (theta_grid % spherical_drop_ratio == 0)
    phi_mask = (phi_grid % spherical_drop_ratio == 0)
    
    # Combine the valid_mask from earlier with spherical mask
    combined_mask = valid_mask & theta_mask & phi_mask
    return points[combined_mask]


def ray_dropping_augmentation(points):
    # Step 1: Random beam drop
    points_after_beam_drop = random_beam_drop(points)
    
    # Step 2: Spherical drop
    points_after_spherical_drop = random_spherical_drop(points_after_beam_drop, np.ones(points_after_beam_drop.shape[0], dtype=bool))
    
    return points_after_spherical_drop

# Example usage
augmented_points_final = ray_dropping_augmentation(lidar_points_with_beam_ids)
beam_ids = augmented_points_final[:,3].reshape(-1,1)
visualize_beam_ids3(augmented_points_final[:,:3], beam_ids)